# 🚗 VIGIL-ROUTE - Road Defect Detection Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Persyvan/vigil-route/blob/main/notebooks/Vigil_Route_Demo.ipynb)

**Developed by Persy Maki ND**  
AI-powered road damage detection system using MobileNetV2

---

## 📌 What This Notebook Does

This interactive demo lets you:
- ✅ Upload road images (smartphone photos or dashcam frames)
- ✅ Run AI detection (Pothole, Pavement Deformation, Healthy Road)
- ✅ Get risk assessment based on speed zone
- ✅ View annotated results with confidence scores

**No installation required!** Just run cells step-by-step.

---

## 🎓 Model Information

| Specification | Details |
|---------------|----------|
| **Architecture** | MobileNetV2 (lightweight CNN) |
| **Accuracy** | 87.9% on test set |
| **Classes** | 3 (Pothole, Deformation, Healthy) |
| **Input Size** | 224×224 RGB |
| **Dataset** | 1,584 Montreal road images (Oct-Dec 2025) |

## 📦 Step 1: Install Dependencies

Install required Python packages (takes ~30 seconds).

In [ ]:
!pip install -q tensorflow opencv-python-headless pillow matplotlib numpy

## 🔧 Step 2: Import Libraries

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import display, Image as IPImage, HTML
import io
from PIL import Image

print("✅ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

## ⚙️ Step 3: Configuration

Define model classes and utility functions.

In [ ]:
# Model classes (order matters!)
CLASS_NAMES = ['deformation_chaussee', 'nid_de_poule', 'route_saine']

CLASS_TRANSLATIONS = {
    'deformation_chaussee': 'Pavement Deformation',
    'nid_de_poule': 'Pothole',
    'route_saine': 'Healthy Road'
}

print("✅ Configuration loaded")
print(f"Model detects {len(CLASS_NAMES)} classes:")
for cls in CLASS_NAMES:
    print(f"  • {CLASS_TRANSLATIONS[cls]}")

## 🧠 Step 4: Define Core Functions

In [ ]:
def predict_defect(image, model, threshold=0.40):
    """
    Predict road defect from image.
    
    Args:
        image: OpenCV image (BGR)
        model: Loaded Keras model
        threshold: Confidence threshold
        
    Returns:
        tuple: (defect_class, confidence, all_detections)
    """
    # Preprocess
    img_resized = cv2.resize(image, (224, 224))
    img_array = tf.keras.utils.img_to_array(img_resized)
    img_batch = np.expand_dims(img_array, 0)
    
    # Predict
    predictions = model.predict(img_batch, verbose=0)
    scores = tf.nn.softmax(predictions[0])
    
    # Collect detections
    detections = []
    for idx, score in enumerate(scores):
        class_name = CLASS_NAMES[idx]
        confidence = float(score)
        
        if class_name == 'route_saine' and confidence < 0.80:
            continue
            
        if confidence >= threshold:
            detections.append((class_name, confidence))
    
    detections.sort(key=lambda x: x[1], reverse=True)
    
    if not detections:
        detections = [('route_saine', float(scores[2]))]
    
    return detections[0][0], detections[0][1], detections


def analyze_risk(defect_class, confidence, speed):
    """
    Calculate risk level based on defect, confidence, and speed.
    """
    if defect_class == 'route_saine':
        return "NONE", "No action required"
    
    base_score = 1.0 if defect_class == 'nid_de_poule' else 0.7
    speed_factor = 1.0 + (speed / 50.0) ** 1.2
    danger_score = (confidence * base_score) * speed_factor
    
    if danger_score >= 1.5:
        return "CRITICAL", "Immediate repair required"
    elif danger_score >= 1.0:
        return "HIGH", "Inspection required within 1 week"
    elif danger_score >= 0.7:
        return "MEDIUM", "Monitoring recommended"
    else:
        return "LOW", "Preventive maintenance"


def display_result(image, defect_class, confidence, risk_level, action):
    """
    Display annotated result in Colab.
    """
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Draw detection box
    h, w = img_rgb.shape[:2]
    colors = {
        'CRITICAL': (255, 0, 0),
        'HIGH': (255, 165, 0),
        'MEDIUM': (255, 255, 0),
        'LOW': (0, 255, 0),
        'NONE': (0, 255, 0)
    }
    color = colors.get(risk_level, (255, 255, 255))
    
    if defect_class != 'route_saine':
        margin_x = int(w * 0.2)
        margin_y = int(h * 0.15)
        cv2.rectangle(img_rgb, 
                     (margin_x, margin_y), 
                     (w - margin_x, h - margin_y), 
                     color, 5)
        
        # Label
        label = f"{CLASS_TRANSLATIONS[defect_class]} - {confidence:.1%}"
        cv2.putText(img_rgb, label, 
                   (margin_x + 10, margin_y + 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
    
    # Display
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(f"Detection: {CLASS_TRANSLATIONS[defect_class]} | Risk: {risk_level}", 
             fontsize=16, fontweight='bold')
    plt.show()
    
    # Print details
    print("\n" + "="*60)
    print("🔍 DETECTION RESULTS")
    print("="*60)
    print(f"Defect Type:     {CLASS_TRANSLATIONS[defect_class]}")
    print(f"Confidence:      {confidence:.1%}")
    print(f"Risk Level:      {risk_level}")
    print(f"Recommended Action: {action}")
    print("="*60 + "\n")

print("✅ Core functions defined")

## 📥 Step 5: Upload Model Weights

**Option 1:** Upload your trained model file (`.keras`)  
**Option 2:** Download from a public URL (if hosted on Drive/HuggingFace)

### Option 1: Upload Model File

In [ ]:
print("📤 Click 'Choose Files' and upload your .keras model file")
uploaded = files.upload()

model_filename = list(uploaded.keys())[0]
print(f"\n✅ Model uploaded: {model_filename}")

## 🤖 Step 6: Load AI Model

In [ ]:
print("🔄 Loading AI model...")

try:
    model = tf.keras.models.load_model(model_filename)
    print("\n✅ Model loaded successfully!")
    print(f"   Architecture: MobileNetV2")
    print(f"   Input shape: {model.input_shape}")
    print(f"   Output classes: {len(CLASS_NAMES)}")
except Exception as e:
    print(f"❌ Error loading model: {e}")

## 📸 Step 7: Upload Test Image

Upload a road image (smartphone photo or dashcam frame).

In [ ]:
print("📤 Click 'Choose Files' and upload a road image (.jpg, .png)")
uploaded_images = files.upload()

image_filename = list(uploaded_images.keys())[0]
print(f"\n✅ Image uploaded: {image_filename}")

# Display uploaded image
img = cv2.imread(image_filename)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.axis('off')
plt.title("Uploaded Image", fontsize=14)
plt.show()

## 🚗 Step 8: Configure Speed Zone

Set the speed zone for risk assessment (impacts danger score).

In [ ]:
# Change this value based on your scenario
SPEED = 50  # km/h

print(f"🚗 Speed zone configured: {SPEED} km/h")
print("\n💡 Speed zones:")
print("   • 30 km/h: Residential area")
print("   • 50 km/h: Urban arterial road")
print("   • 70+ km/h: Highway")

## 🎯 Step 9: Run Detection

Analyze the image and get AI prediction with risk assessment.

In [ ]:
print("🤖 Running AI detection...\n")

# Predict
defect_class, confidence, all_detections = predict_defect(img, model)

# Analyze risk
risk_level, action = analyze_risk(defect_class, confidence, SPEED)

# Display results
display_result(img, defect_class, confidence, risk_level, action)

# Show all detections if multiple
if len(all_detections) > 1:
    print("ℹ️  All detected defects:")
    for cls, conf in all_detections:
        print(f"   • {CLASS_TRANSLATIONS[cls]}: {conf:.1%}")

## 📦 (Optional) Step 10: Batch Processing

Upload multiple images at once for batch analysis.

In [ ]:
print("📤 Upload multiple images for batch processing")
batch_uploaded = files.upload()

print(f"\n✅ {len(batch_uploaded)} images uploaded")
print("\nProcessing...\n")

results = []

for idx, filename in enumerate(batch_uploaded.keys(), 1):
    print(f"📷 Image {idx}/{len(batch_uploaded)}: {filename}")
    
    img = cv2.imread(filename)
    defect_class, confidence, _ = predict_defect(img, model)
    risk_level, action = analyze_risk(defect_class, confidence, SPEED)
    
    print(f"   Detected: {CLASS_TRANSLATIONS[defect_class]} ({confidence:.1%})")
    print(f"   Risk: {risk_level}\n")
    
    results.append({
        'image': filename,
        'defect': CLASS_TRANSLATIONS[defect_class],
        'confidence': confidence,
        'risk': risk_level
    })

print("\n" + "="*60)
print("📊 BATCH SUMMARY")
print("="*60)
critical = sum(1 for r in results if r['risk'] == 'CRITICAL')
high = sum(1 for r in results if r['risk'] == 'HIGH')
print(f"Total images: {len(results)}")
print(f"Critical:     {critical}")
print(f"High:         {high}")
print("="*60)

---

## 🎉 Demo Complete!

### Next Steps

- 📖 [Full Documentation](https://github.com/Persyvan/vigil-route)
- 📧 Contact: [persy.maki.ml@gmail.com](mailto:persy.maki.ml@gmail.com)
- 💼 [LinkedIn](https://linkedin.com/in/persy-maki)
- 🐙 [GitHub](https://github.com/Persyvan)

---

**© 2026 Persy Maki ND - VIGIL-ROUTE**  
*AI-Powered Road Infrastructure Monitoring*